# Проект: Определение стоимости автомобилей

## Постановка задачи (кратко)

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Метрика конечной модели RMSE не должна превышать 2500.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

from sklearn.metrics import mean_squared_error

Загрузим и изучим данные:

In [2]:
df = pd.read_csv('/datasets/autos.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [3]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


В данных присутствуют категориальные и количественные признаки, есть пропуски и аномалии, необходима предобработка. Выполним её:

In [5]:
df.Price.value_counts().head(5)

0       10772
500      5670
1500     5394
1000     4649
1200     4594
Name: Price, dtype: int64

In [6]:
df[df['Price'] < 10].Price.value_counts()

0    10772
1     1189
5       26
2       12
8        9
9        8
3        8
7        3
4        1
Name: Price, dtype: int64

Найдены аномально низкие цены в целевом признаке. Действительно ли авто отдаются даром или за символическую плату - не известно. Пока оставим как есть, продолжим EDA.

Для удобства заменим пропуски в категориальных признаках на *unknown*:

In [7]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
num_features = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']
exc_features = ['DateCrawled', 'DateCreated', 'NumberOfPictures', 'PostalCode', 'LastSeen']

In [8]:
df = df.fillna(value=dict(zip(cat_features, ['unknown'] * len(cat_features))))

Столбцы модели и марки автомобиля тесно связаны. Объединим их в один признак:

In [9]:
df['Model'] = df['Brand'] + ' ' + df['Model']
df = df.drop('Brand', axis=1)
cat_features.remove('Brand')

Столбцы года и месяца регистрации также объединим в один:

In [10]:
df.loc[df['RegistrationMonth'] == 0, 'RegistrationMonth'] = 1
df['RegistrationYear'] = df['RegistrationYear'] + (df['RegistrationMonth'] - 1) / 12
df = df.drop('RegistrationMonth', axis=1)
num_features.remove('RegistrationMonth')

Остальные количественные признаки намеренно **изменять не будем**: если такие неоднозначные данные пришли в продакшн, то модель всё равно должна уметь работать с аномальными данными.

Отбросим неинформативные столбцы:

In [11]:
df = df.drop(exc_features, axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Price             354369 non-null  int64  
 1   VehicleType       354369 non-null  object 
 2   RegistrationYear  354369 non-null  float64
 3   Gearbox           354369 non-null  object 
 4   Power             354369 non-null  int64  
 5   Model             354369 non-null  object 
 6   Kilometer         354369 non-null  int64  
 7   FuelType          354369 non-null  object 
 8   Repaired          354369 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 24.3+ MB


Пропусков не осталось, но есть подозрения на пропущенные аномалии. Проведем анализ тщательнее с использованием дополнительных библиотек.

In [13]:
pip install pandas-profiling[notebook]

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas_profiling

/tmp/ipykernel_854/1591302161.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [15]:
#pandas_profiling.ProfileReport(df)

Из отчета видны аномальные значения в RegistrationYear и Power. Эти признаки кореллируют с целевым, поэтому избавимся от аномалий.

Машины из будущего и альтернативного прошлого исключим из анализа:

In [16]:
df[df['RegistrationYear'] > 2020]['RegistrationYear'].sort_values().count()

105

In [17]:
df[df['RegistrationYear'] < 1900]['RegistrationYear'].sort_values().count()

66

In [18]:
df = df.drop(df[df['RegistrationYear'] > 2020].index)
df = df.drop(df[df['RegistrationYear'] < 1900].index)

Карьерный самосвал (самый мощный автомобиль) имеет мощность 4000 л.с., а автомобили начала XX-го века - не менее 10:

In [19]:
df[df['Power'] > 4000]['Power'].sort_values().count()

87

In [20]:
df[df['Power'] < 10]['Power'].sort_values().count()

40317

In [21]:
df = df.drop(df[df['Power'] > 4000].index)
df = df.drop(df[df['Power'] < 10].index)

Также избавимся от всех записей со странными ценами:

In [22]:
df[df['Price'] < 10]['Price'].sort_values().count()

7607

In [23]:
df = df.drop(df[df['Price'] < 10].index)

In [24]:
df.shape

(306187, 9)

## Обучение моделей

### Подготовка данных

Подготовим данные для обучения моделей. 

Разобьем категориальные признаки техникой OHE:

In [25]:
df = pd.get_dummies(df, drop_first=True)

Разделим данные на обучающую, валидационную и тестовую выборки:

In [26]:
features = df.drop('Price', axis=1)
target = df['Price']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=42)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=42)

Стандартизируем количественные признаки:

In [27]:
scaler = StandardScaler()
scaler.fit(features_train[num_features])

features_train_scaled = pd.DataFrame(scaler.transform(features_train[num_features]), columns=num_features)
features_valid_scaled = pd.DataFrame(scaler.transform(features_valid[num_features]), columns=num_features)
features_test_scaled = pd.DataFrame(scaler.transform(features_test[num_features]), columns=num_features)

features_train = features_train_scaled.join(features_train.drop(num_features, axis=1).reset_index(drop=True))
features_valid = features_valid_scaled.join(features_valid.drop(num_features, axis=1).reset_index(drop=True))
features_test = features_test_scaled.join(features_test.drop(num_features, axis=1).reset_index(drop=True))

target_train = target_train.reset_index(drop=True)
target_valid = target_valid.reset_index(drop=True)
target_test = target_test.reset_index(drop=True)

Далее обучим модели.

### Модель №1

In [28]:
# LightGBM случайный лес, обученный градиентным бустингом

lgb_train = lgb.Dataset(features_train, target_train)
lgb_valid = lgb.Dataset(features_valid, target_valid, reference=lgb_train)

In [29]:
%%time

params = {'metric' : 'rmse',
          'num_leaves' : 15,
          'max_depth' : 5,
          'learning_rate' : 0.1,
          'n_estimators' : 5
}

model_lgb = lgb.train(params, lgb_train, valid_sets=[lgb_valid])

/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.676166 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1160
[LightGBM] [Info] Number of data points in the train set: 183711, number of used features: 321
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4821.436011
[1]	valid_0's rmse: 4280.34
[2]	valid_0's rmse: 4022.28
[3]	valid_0's rmse: 3799.25
[4]	valid_0's rmse: 3586.83
[5]	valid_0's rmse: 3407.64
CPU times: user 15.2 s, sys: 456 ms, total: 15.6 s
Wall time: 15.7 s


Попробуем изменить параметры модели:

In [30]:
%%time

# повысим макс. число листьев на одном дереве (num_leaves)
params = {'metric' : 'rmse',
          'num_leaves' : 31,
          'max_depth' : 5,
          'learning_rate' : 0.1,
          'n_estimators' : 5
}

model_lgb = lgb.train(params, lgb_train, valid_sets=[lgb_valid])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.594699 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1160
[LightGBM] [Info] Number of data points in the train set: 183711, number of used features: 321
[LightGBM] [Info] Start training from score 4821.436011
[1]	valid_0's rmse: 4257.93
[2]	valid_0's rmse: 3977.71
[3]	valid_0's rmse: 3730.16
[4]	valid_0's rmse: 3511.32
[5]	

In [31]:
%%time

# увеличим коэф. скорости обучения (learning_rate)
params = {'metric' : 'rmse',
          'num_leaves' : 31,
          'max_depth' : 5,
          'learning_rate' : 0.3,
          'n_estimators' : 5
}

model_lgb = lgb.train(params, lgb_train, valid_sets=[lgb_valid])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.673639 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1160
[LightGBM] [Info] Number of data points in the train set: 183711, number of used features: 321
[LightGBM] [Info] Start training from score 4821.436011
[1]	valid_0's rmse: 3660.04
[2]	valid_0's rmse: 3072.03
[3]	valid_0's rmse: 2680.21
[4]	valid_0's rmse: 2440.47
[5]	

In [32]:
%%time

# добавим макс. глубины деревьям (max_depth)
params = {'metric' : 'rmse',
          'num_leaves' : 31,
          'max_depth' : 10,
          'learning_rate' : 0.3,
          'n_estimators' : 5
}

model_lgb = lgb.train(params, lgb_train, valid_sets=[lgb_valid])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.503797 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1160
[LightGBM] [Info] Number of data points in the train set: 183711, number of used features: 321
[LightGBM] [Info] Start training from score 4821.436011
[1]	valid_0's rmse: 3635.24
[2]	valid_0's rmse: 3030.28
[3]	valid_0's rmse: 2650.62
[4]	valid_0's rmse: 2399.51
[5]	

In [33]:
%%time

# увеличим число деревьев (n_estimators)
params = {'metric' : 'rmse',
          'num_leaves' : 31,
          'max_depth' : 10,
          'learning_rate' : 0.3,
          'n_estimators' : 10
}

model_lgb = lgb.train(params, lgb_train, valid_sets=[lgb_valid])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.588304 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1160
[LightGBM] [Info] Number of data points in the train set: 183711, number of used features: 321
[LightGBM] [Info] Start training from score 4821.436011
[1]	valid_0's rmse: 3635.24
[2]	valid_0's rmse: 3030.28
[3]	valid_0's rmse: 2650.62
[4]	valid_0's rmse: 2399.51
[5]	

In [34]:
%%time
pred_lgb = model_lgb.predict(features_valid)

CPU times: user 218 ms, sys: 93.7 ms, total: 311 ms
Wall time: 299 ms


### Модель №2

In [35]:
# Случайный лес с параметрами предыдущей модели

model_forest = RandomForestRegressor(max_leaf_nodes=31,
                                     max_depth=10,
                                     n_estimators=10,
                                     random_state=42)

In [36]:
%%time
model_forest.fit(features_train, target_train)

CPU times: user 19.3 s, sys: 224 ms, total: 19.6 s
Wall time: 19.6 s


RandomForestRegressor(max_depth=10, max_leaf_nodes=31, n_estimators=10,
                      random_state=42)

In [37]:
%%time
pred_forest = model_forest.predict(features_valid)

CPU times: user 83.1 ms, sys: 53.6 ms, total: 137 ms
Wall time: 146 ms


In [38]:
mean_squared_error(target_valid, pred_forest, squared=False)

2383.5138749591356

### Модель №3

In [39]:
# Линейная модель, обученная стохастическим градиентным спуском

model_sgdregr = SGDRegressor(max_iter=1000, shuffle=True, random_state=42)

In [40]:
%%time
model_sgdregr.fit(features_train, target_train)

CPU times: user 14.7 s, sys: 275 ms, total: 15 s
Wall time: 15 s


SGDRegressor(random_state=42)

In [41]:
%%time
pred_sgdregr = model_sgdregr.predict(features_valid)

CPU times: user 59.1 ms, sys: 68.9 ms, total: 128 ms
Wall time: 89.6 ms


In [42]:
mean_squared_error(target_valid, pred_sgdregr, squared=False)

2725.934638262295

### Модель №4

In [43]:
# Обычная линейная регрессия

model_regr = LinearRegression()

In [44]:
%%time
model_regr.fit(features_train, target_train)

CPU times: user 12.8 s, sys: 5.19 s, total: 18 s
Wall time: 18 s


LinearRegression()

In [45]:
%%time
pred_regr = model_regr.predict(features_valid)

CPU times: user 64.9 ms, sys: 68.8 ms, total: 134 ms
Wall time: 102 ms


In [46]:
mean_squared_error(target_valid, pred_regr, squared=False)

2708.428009913883

## Анализ моделей

Критерии оценивания моделей:
- качество предсказания;
- время обучения модели;
- время предсказания модели.

In [50]:
labels = ['RMSE', 'Время обучения, сек', 'Время предсказания, сек']
rows = [[1959.66, 24.6, 0.299],
        [2383.51, 19.6, 0.146],
        [2725.93, 15, 0.09],
        [2708.42, 18, 0.102]
       ]
index = ['LightGBM лес', 'sklearn лес', 'sklearn SGD регрессия', 'sklearn лин. регрессия']
pd.DataFrame(data=rows, columns=labels, index=index)

,RMSE,"Время обучения, сек","Время предсказания, сек"
LightGBM лес,1959.66,24.6,0.299
sklearn лес,2383.51,19.6,0.146
sklearn SGD регрессия,2725.93,15.0,0.090
sklearn лин. регрессия,2708.42,18.0,0.102


LightGBM лес показал наилучшие параметры в точности, средние в скорости обучения и худшие во времени предсказания. Тем не менее, время предсказания для конечного пользователя меньше секунды и пренебрежимо мало.

Проведем окончательную проверку для наилучшей модели на тестовой выборке:

In [48]:
lgb_test = lgb.Dataset(features_test, target_test)

lgb.train(params, lgb_train, valid_sets=[lgb_test])

/opt/conda/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.284941 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1160
[LightGBM] [Info] Number of data points in the train set: 183711, number of used features: 321
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4821.436011
[1]	valid_0's rmse: 3651.5
[2]	valid_0's rmse: 3049.14
[3]	valid_0's rmse: 2669.43
[4]	valid_0's rmse: 2421.19
[5]	valid_0's rmse: 2270.12
[6]	valid_0's rmse: 2166.06
[7]	valid_0's

Значение метрики RMSE не превышает порог в 2500. Модель подтвердила свою точность на тестовой выборке, её можно рекомендовать к внедрению.

## Выводы

### Результаты

Cоздан прототип модели машинного обучения, определяющей стоимость автомобиля. 
- Лучшая модель: LightGBM
- Параметры модели: num_leaves=31, max_depth=10, n_estimators=10, learning_rate=0.3
- Проверка на тестовых данных: RMSE=1959.66, время обучения = 24.6 сек, время предсказания = 0.299 сек

### Что можно улучшить

1. Визуализировать важность факторов при моделировании: показать feature_importances_ модели.
2. Feature Engineering: у разных машин разный возраст, что может влиять на ценообразование.